# App Market Research

In this project, publicly available data about dog play-date apps from Google Play Store is scrapped and analyzed to access the potential of such doggy play-date apps in the app market. Possible insights from the analysis might include:
* What do people love most and least about such apps?
* How well do such apps generally tend to do financially?
* What regions of the world are such apps currently found in?

In [86]:
# Imports.
import pycountry
from google_play_scraper import search
from utility_functions import inspect_function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
# 20 Countries based on dogs per capita.
# References: 
# 1. https://www.mappr.co/thematic-maps/world-pet-ownership/
# 2. https://www.petsecure.com.au/pet-care/a-guide-to-worldwide-pet-ownership/

country_names = [ # ISO standard names.
    'United States', 'Brazil', 'China', 'Russian Federation', 'Japan', 
    'Philippines', 'India', 'Argentina', 'United Kingdom', 'France', 
    'Poland', 'Spain', 'Romania', 'Australia', 'Hungary', 
    'Czechia', 'South Africa', 'Germany', 'Ethiopia',  'Canada'
] 

## 1. Data Acquisition

In [104]:
inspect_function(search)


FUNCTION search(...):
1. query: str
2. n_hits: int = 30
3. lang: str = 'en'
4. country: str = 'us'


In [111]:
search_str = "dog play date" # Search string => kind of app we're interested in.
country_code = pycountry.countries.get(name=country_names[0]).alpha_2.lower()
result = search(
    search_str,
    lang="en",  # English (defaults to 'en').
    country=country_code,  # (defaults to 'us').
    n_hits=30  # defaults to 30 (= Google's maximum)
)

The above result is comprised of a list of objects with the following fields.
1. `appId`: Unique identifier.
2. `icon`: Link to app icon image.
3. `screenshots`: List of links to screenshots of the app.
4. `title`: Brief title.
5. `score`: Average rating out of 5. `!!!`
6. `genre`: App type category.
7. `price`: Cost of installation.
8. `free`: Whether or not app is free.
9. `currency`: Currency in which the price is expressed.
10. `video`: A video associated with this app.
11. `videoImage`: Link to a thumbnail image of above video.
12. `description`: Description in plain text format.
13. `descriptionHTML`: Description in HTML format.
14. `developer`: Person/entity that has developed this app.
15. `installs`: No. of installs.

In [113]:
result[0]

{'appId': 'com.colin.barkerchat',
 'icon': 'https://play-lh.googleusercontent.com/XAM-J1LP6bakE_qM5mYoxYuNF86aWi1zIzjff2HKlRAaN2x4FrIZJVpgegv2cKHp-Ec',
 'screenshots': ['https://play-lh.googleusercontent.com/d5bZ6Frv7I1pxMadPFJsbZ8TUJNkb22OB8eqFOXdxGIiJcwZw2TiiVvSBsA6q7HLmw',
  'https://play-lh.googleusercontent.com/9G8DE0ezIBrf-8gkjASjP2AoUbFoSXwkouG6U2W0F8DVn-6Iq7jFPGS3AhQYps1z_NeD',
  'https://play-lh.googleusercontent.com/DT6bJCwZEPnCYWCpfjazGM0E7DhoXcs2OORQ6-QBxJgSB80soLd5vgxda-JbfUc7L9g',
  'https://play-lh.googleusercontent.com/pVVfLh3ygnk5htEElN32N00D8p54cHnC4lMY4zM7To78sxtET3RUW5v_m66lKzzfLg',
  'https://play-lh.googleusercontent.com/hCcsgXTluMIEi_ZV4CMtOx9AC_SFl6GberXzLBG5MuvM_aoO5tEksejg5U0PsNn9vA',
  'https://play-lh.googleusercontent.com/EuqwuNYw_rl1MvrOY3BaDNA44AnslrRu3b_imv0M9WJ2P9xgII48qIlhyN6fbrvOnZqL',
  'https://play-lh.googleusercontent.com/7ikW5DDM3bqViGrdIITeaG2-y-QtoVmnUYEfpE8ZzSCspTzeLTdNfsWu0dqFA-x8GM__',
  'https://play-lh.googleusercontent.com/fbra0vjhQpZxxWq